In [1]:
import numpy as np 
# Epydemix import
import sys
sys.path.append("../")
from epydemix.epimodel import EpiModel
from epydemix.plotting import plot_quantiles, plot_spectral_radius
from epydemix.utils import create_definitions, convert_to_2Darray, evaluate, compute_days

In [2]:
start_date, end_date = "2019-12-01", "2020-04-01"
T = compute_days(start_date, end_date)

# Simulation
# create model 
model = EpiModel(compartments=["S", "I", "R"], population_data_path="../epydemix_data/Indonesia/", population_name="Indonesia")

# define parameters
model.add_parameter(name="beta", value=0.03)
model.add_parameter(name="mu", value=0.2)

# add transitions 
model.add_transition(source="S", target="I", rate="beta", agent="I")
model.add_transition(source="I", target="R", rate="mu")

# model definitions 
model.definitions = create_definitions(model.parameters, T, n_age=model.population.Nk.shape[0])
model.definitions["beta"]

array([[0.03, 0.03, 0.03, ..., 0.03, 0.03, 0.03],
       [0.03, 0.03, 0.03, ..., 0.03, 0.03, 0.03],
       [0.03, 0.03, 0.03, ..., 0.03, 0.03, 0.03],
       ...,
       [0.03, 0.03, 0.03, ..., 0.03, 0.03, 0.03],
       [0.03, 0.03, 0.03, ..., 0.03, 0.03, 0.03],
       [0.03, 0.03, 0.03, ..., 0.03, 0.03, 0.03]])

In [3]:
results = model.run_simulations(start_date=start_date, 
                                end_date=end_date, 
                                S=model.population.Nk - np.ones(len(model.population.Nk)),
                                I=np.ones(len(model.population.Nk)),
                                steps="daily", 
                                Nsim=1)

[0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03
 0.03 0.03]
[0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03
 0.03 0.03]
[0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03
 0.03 0.03]
[0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03
 0.03 0.03]
	 0 0.03
	 1 1.6700566621574223e-08
	 2 1.6700566621574223e-08
	 3 1.6700566621574223e-08
	 4 1.6700566621574223e-08
	 5 1.6700566621574223e-08
	 6 1.6700566621574223e-08
	 7 1.6700566621574223e-08
	 8 1.6700566621574223e-08
	 9 1.6700566621574223e-08
	 10 1.6700566621574223e-08
	 11 1.6700566621574223e-08
	 12 1.6700566621574223e-08
	 13 1.6700566621574223e-08
	 14 1.6700566621574223e-08
	 15 1.6700566621574223e-08
	 16 1.6700566621574223e-08
	 17 1.6700566621574223e-08
	 18 1.6700566621574223e-08
	 19 1.6700566621574223e-08
	 20 1.6700566621574223e-08
	 21 1.6700566621574223e-08
	 22 1.6700566621574223e-08
	 23 1.6700566621574223e-08
	 24 1.6700566621574223e

In [ ]:
plot_quantiles(results, columns=["I_total", "S_total", "R_total"])

In [ ]:
model.definitions

In [ ]:
# create model 
model = EpiModel(compartments=["S", "I", "R"], population_data_path="../epydemix_data/Indonesia/", population_name="Indonesia")

In [ ]:
# Scalar parameters
model.add_parameter(name="beta", value=0.04)
definitions = create_definitions(model.parameters, T=100, n_age=model.population.Nk.shape[0])

print(definitions["beta"].shape)
definitions

In [ ]:
# Time-varying parameter with wrong length
model.add_parameter(name="beta", value=np.random.uniform(0.01, 0.1, 80))
definitions = create_definitions(model.parameters, T=100, n_age=model.population.Nk.shape[0])

print(definitions["beta"].shape)
definitions

In [ ]:
# Time-varying parameter with correct length
model.add_parameter(name="beta", value=np.random.uniform(0.01, 0.1, 100))
definitions = create_definitions(model.parameters, T=100, n_age=model.population.Nk.shape[0])

print(definitions["beta"].shape)
definitions

In [ ]:
# Age-varying parameter with correct length
beta = np.random.uniform(0.01, 0.1, 16)
beta = convert_to_2Darray(beta)   # equivalent to np.random.uniform(0.01, 0.1, (1, 16))
model.add_parameter(name="beta", value=beta)
definitions = create_definitions(model.parameters, T=100, n_age=model.population.Nk.shape[0])

print(definitions["beta"].shape)
definitions

In [ ]:
# Age and time-varying parameter with correct length
model.add_parameter(name="beta", value= np.random.uniform(0.01, 0.1, size=(100, 16)))
definitions = create_definitions(model.parameters, T=100, n_age=model.population.Nk.shape[0])

print(definitions["beta"].shape)
definitions

In [ ]:
model.definitions = create_definitions(model.parameters, T=100, n_age=model.population.Nk.shape[0])
model.definitions["beta"]

In [ ]:
initial_definitions = create_definitions(model.parameters, T=100, n_age=model.population.Nk.shape[0])
np.any(model.definitions["beta"] != initial_definitions["beta"])

In [ ]:
model.definitions["beta"][0]

In [ ]:
# Before the loop
parameters = model.definitions
import copy
original_parameters = copy.deepcopy(parameters)
print(f"Original parameters at start: {original_parameters}")

# Inside the loop after calling `evaluate`
for tr in model.transitions["S"]:
    rate = evaluate(expr=tr.rate, env=parameters)
    print(f"Parameters after evaluate: {parameters}")

